In [1]:
import pandas as pd
#df = pd.read_csv('../dataset/wine.csv', header=None)
df_pre = pd.read_csv('../dataset/wine.csv', 
                 names=['주석산 농도', '아세트산 농도', '구연산 농도', '잔류 당분 농도', '염화나트륨 농도', '유리 아황산 농도', '총 아황산 농도', '밀도', 'pH', '황산칼륨 농도', 
                        '알코올 도수', '와인의 맛', 'class'])
# df = df_pre.sample(frac=1.0)
df = df_pre

In [2]:
#print(df.head())
print(df.info())
#print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   주석산 농도     6497 non-null   float64
 1   아세트산 농도    6497 non-null   float64
 2   구연산 농도     6497 non-null   float64
 3   잔류 당분 농도   6497 non-null   float64
 4   염화나트륨 농도   6497 non-null   float64
 5   유리 아황산 농도  6497 non-null   float64
 6   총 아황산 농도   6497 non-null   float64
 7   밀도         6497 non-null   float64
 8   pH         6497 non-null   float64
 9   황산칼륨 농도    6497 non-null   float64
 10  알코올 도수     6497 non-null   float64
 11  와인의 맛      6497 non-null   int64  
 12  class      6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB
None


In [3]:
df[['주석산 농도', 'class']]

,주석산 농도,class
0,7.4,1
1,7.8,1
2,7.8,1
3,11.2,1
4,7.4,1
...,...,...
6492,6.2,0
6493,6.6,0
6494,6.5,0
6495,5.5,0


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import os
print(tf.__version__)

# seed값 생성
np.random.seed(3)
tf.random.set_seed(3)

X = df.drop(['class'], axis=1, inplace=False).values  # inplacebool, default False If False, return a copy. Otherwise, do operation inplace and return None.
Y = df[['class']].values

2.1.0


In [5]:
def build_model():
    model = Sequential([
        Dense(30, input_shape=(12,), activation='relu'),
        Dense(12, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1)
    ])
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam(1e-4)
    model.compile(loss=loss,
                 optimizer=optimizer,
                 metrics=['accuracy'])
    return model

model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                390       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                372       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 875
Trainable params: 875
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 모델 저장 폴더 만들기
model_dir = './saved_model_tf/'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + "{epoch:04d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(
    filepath=model_path, 
    monitor='val_loss',         
    # mode='min',        
    verbose=1, # verbosity mode, 0 or 1.
    # save_best_only=True,
    save_weights_only=True,    
    #save_freq='epoch',
    period=50)
    
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50, baseline=0.4)

In [7]:
history = model.fit(X, Y,
                   validation_split=0.2,
                   epochs=3500,
                   batch_size = 128,
                   verbose=1,  # Verbosity mode. 0 = silent, 1 = progress bar(default), 2 = one line per epoch. 
                   callbacks = [checkpointer, early_stopping_callback])

Train on 5197 samples, validate on 1300 samples
Epoch 1/3500
5197/5197 [==============================] - 1s 164us/sample - loss: 1.7674 - accuracy: 0.6923 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 2/3500
5197/5197 [==============================] - 0s 28us/sample - loss: 0.8695 - accuracy: 0.6917 - val_loss: 0.2750 - val_accuracy: 0.9785
Epoch 3/3500
5197/5197 [==============================] - 0s 32us/sample - loss: 0.6260 - accuracy: 0.6919 - val_loss: 0.2003 - val_accuracy: 0.9877
Epoch 4/3500
5197/5197 [==============================] - 0s 26us/sample - loss: 0.5555 - accuracy: 0.6925 - val_loss: 0.2050 - val_accuracy: 0.9877
Epoch 5/3500
5197/5197 [==============================] - 0s 26us/sample - loss: 0.4991 - accuracy: 0.6956 - val_loss: 0.2110 - val_accuracy: 0.9862
Epoch 6/3500
5197/5197 [==============================] - 0s 27us/sample - loss: 0.4533 - accuracy: 0.7079 - val_loss: 0.1920 - val_accuracy: 0.9892
Epoch 7/3500
5197/5197 [==============================] -

In [8]:
'''Weight들만 저장하기'''
model.save_weights('./checkpoints_tf/my_checkpoint')  # 수동으로 가중치 저장하기

'''모델 전체 저장하기'''
model.save(model_dir) # 전체 모델 저장하기

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./saved_model_tf/assets


In [9]:
'''모델 전체 불러오기'''
new_model = tf.keras.models.load_model('./saved_model_tf') 

'''모델 구조를 확인'''
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                390       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                372       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 875
Trainable params: 875
Non-trainable params: 0
_________________________________________________________________


In [10]:
# 복원된 모델 평가
loss, acc = new_model.evaluate(X, Y, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))
print(new_model.predict(X).shape)

6497/6497 - 0s - loss: 0.2051 - accuracy: 0.9286
복원된 모델의 정확도: 92.86%
(6497, 1)


In [11]:
# 모델 weight 불러오기
checkpoint_path = "checkpoints_tf/my_checkpoint"
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(X, Y, verbose=2)
print(loss)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

print(model.predict(X).shape)

6497/6497 - 0s - loss: 0.2051 - accuracy: 0.9286
0.2050594309279235
복원된 모델의 정확도: 92.86%
(6497, 1)
